@author: Diego CASAS BARCENAS, Romain SALMERON, Leo Jean UNITE

# Étape 1 : Chargement et exploration des données
On va commencer par :

1. Charger le fichier CSV dans un notebook Jupyter.
2. Explorer les colonnes pour identifier les données utiles à la recherche des coordonnées (adresse, code postal, ville, etc.).
3. Vérifier la qualité des données (valeurs manquantes, incohérences).

In [17]:
import pandas as pd

# Charger le fichier CSV
file_path = 'data/BDCOM_2023(in).csv'
df = pd.read_csv(file_path, encoding='ISO-8859-1')

# Aperçu des premières lignes
print(df.head())

# Vérification des colonnes disponibles
print(df.columns)

# Vérification des valeurs manquantes
print(df.isnull().sum())

                  X             Y  OBJECTID   c_ord  arro  qua           xbis  \
0  651791.048600003  6.862992e+06       1.0  1311.0   1.0  2.0  651792.345590   
1       652152.0612  6.862579e+06       2.0  1464.0   1.0  2.0  652152.061200   
2  651430.135700002  6.862714e+06       4.0  1623.0   1.0  3.0  651430.135700   
3  651133.490999997  6.862932e+06       6.0  2087.0   1.0  3.0  651130.053214   
4  651124.613200001  6.863066e+06       7.0  2157.0   1.0  3.0  651124.613200   

           ybis    num  let  ... codact                   ens  bio surf cc_id  \
0  6.862996e+06   25.0  NaN  ...  CB107                   Y'S  0.0  1.0   0.0   
1  6.862579e+06    1.0  NaN  ...  CC301        ALAIN AFFLELOU  0.0  1.0   1.0   
2  6.862714e+06  196.0  NaN  ...  CH106         ENZA FAMIGLIA  0.0  1.0   0.0   
3  6.862939e+06    7.0  NaN  ...  SA202          JULIE BEAUTE  0.0  1.0   0.0   
4  6.863066e+06   20.0  NaN  ...  CD201  HOME AUTOUR DU MONDE  0.0  1.0   0.0   

  cc_niv    niv47  niv18  

## Supposons que l'adresse complète soit dans une colonne "adresse"

In [18]:
# Créer un fichier CSV avec uniquement les adresses à géocoder
df['adresse'] = df['num'].astype(str) + ' ' + df['let'].fillna('') + ' ' + df['typ_voie'] + ' ' + df['lib_voie'] + ', Paris ' + df['arro'].astype(str)
df[['adresse']].to_csv("data/export_for_search.csv", index=False, encoding='ISO-8859-1')

# Étape 2 : Envoyer les adresses à l'API

In [19]:
import requests

# URL de l'API du gouvernement français pour le géocodage
url = "https://api-adresse.data.gouv.fr/search/csv/"

# Ouvrir le fichier contenant les adresses
with open("data/export_for_search.csv", "rb") as fichier:
    response = requests.post(url, files={"data": fichier})

# Vérifier si la requête a réussi
if response.status_code == 200:
    # Sauvegarder le fichier géocodé
    with open("data/result_geocoded.csv", "wb") as output:
        output.write(response.content)
    print("Géocodage réussi. Fichier sauvegardé sous 'data/result_geocoded.csv'.")
else:
    print("Erreur lors du géocodage :", response.status_code)


Géocodage réussi. Fichier sauvegardé sous 'data/result_geocoded.csv'.


# Étape 3 : Intégrer les coordonnées dans le dataset initial

In [20]:
# Charger les résultats géocodés
df_geocoded = pd.read_csv("data/result_geocoded.csv", encoding='ISO-8859-1')

# Fusionner avec les données originales sur la colonne "adresse"
df_final = df.merge(df_geocoded[['adresse', 'latitude', 'longitude']], on='adresse', how='left')

# Sauvegarder le fichier final enrichi
df_final.to_csv("data/BCOM2023_enriched.csv", index=False, encoding='ISO-8859-1')
print("Fichier enrichi avec coordonnées sauvegardé.")

Fichier enrichi avec coordonnées sauvegardé.


# Étape 4 : Rechercher les sites web des commerces
1. Analyser les colonnes disponibles : Vérifiez si l'une des colonnes contient déjà des informations sur le site web des commerces.
2. Compléter les données manquantes : Si le site web est manquant, vous pouvez soit utiliser un service de recherche automatisée (comme `Google Places API`) ou un autre service d'API pour récupérer ces informations, mais attention aux quotas d'usage.

## 1. Installer les librairies nécessaires
`pip install pandas selenium beautifulsoup4`

In [21]:
import pandas
from selenium import webdriver
from bs4 import BeautifulSoup
import time

### Chargement des données

In [22]:
df = pandas.read_csv("data/BCOM2023_enriched.csv", encoding='ISO-8859-1')
df

C:\Users\leoun\AppData\Local\Temp\ipykernel_11896\2588200154.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pandas.read_csv("data/BCOM2023_enriched.csv", encoding='ISO-8859-1')


,X,Y,OBJECTID,c_ord,arro,qua,xbis,ybis,num,let,...,surf,cc_id,cc_niv,niv47,niv18,niv8,niv2,adresse,latitude,longitude
0,651791.048600003,6.862992e+06,1.0,1311.0,1.0,2.0,651792.34559,6.862996e+06,25.0,NaN,...,1.0,0.0,NaN,10301.0,103.0,3.0,1.0,"25.0 RUE LOUVRE, Paris 1.0",49.042280,2.507517
1,652152.0612,6.862579e+06,2.0,1464.0,1.0,2.0,652152.06120,6.862579e+06,1.0,NaN,...,1.0,1.0,-3.0,10403.0,104.0,3.0,1.0,"1.0 RUE PIERRE LESCOT, Paris 1.0",48.862383,2.348379
2,652152.0612,6.862579e+06,2.0,1464.0,1.0,2.0,652152.06120,6.862579e+06,1.0,NaN,...,1.0,1.0,-3.0,10403.0,104.0,3.0,1.0,"1.0 RUE PIERRE LESCOT, Paris 1.0",48.862383,2.348379
3,652152.0612,6.862579e+06,2.0,1464.0,1.0,2.0,652152.06120,6.862579e+06,1.0,NaN,...,1.0,1.0,-3.0,10403.0,104.0,3.0,1.0,"1.0 RUE PIERRE LESCOT, Paris 1.0",48.862383,2.348379
4,652152.0612,6.862579e+06,2.0,1464.0,1.0,2.0,652152.06120,6.862579e+06,1.0,NaN,...,1.0,1.0,-3.0,10403.0,104.0,3.0,1.0,"1.0 RUE PIERRE LESCOT, Paris 1.0",48.862383,2.348379
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206718,654109.7963,6.862549e+06,83149.0,96027.0,11.0,42.0,654109.79630,6.862549e+06,71.0,NaN,...,1.0,0.0,NaN,10401.0,104.0,3.0,1.0,"71.0 BD VOLTAIRE, Paris 11.0",48.852436,2.391346
206719,654109.7963,6.862549e+06,83149.0,96027.0,11.0,42.0,654109.79630,6.862549e+06,71.0,NaN,...,1.0,0.0,NaN,10401.0,104.0,3.0,1.0,"71.0 BD VOLTAIRE, Paris 11.0",48.852436,2.391346
206720,654109.7963,6.862549e+06,83149.0,96027.0,11.0,42.0,654109.79630,6.862549e+06,71.0,NaN,...,1.0,0.0,NaN,10401.0,104.0,3.0,1.0,"71.0 BD VOLTAIRE, Paris 11.0",48.852436,2.391346
206721,652951.749,6.858210e+06,83150.0,96057.0,13.0,51.0,652951.74900,6.858210e+06,33.0,NaN,...,1.0,0.0,NaN,11101.0,111.0,5.0,1.0,"33.0 RUE CAILLAUX, Paris 13.0",48.822594,2.360637


## 2. Télécharger et configurer ChromeDriver
Pour que `selenium` fonctionne comme ce qui suit, sous Windows, il est nécessaire d’avoir télécharger l’exécutable [chromedriver](https://storage.googleapis.com/chrome-for-testing-public/132.0.6834.110/win64/chromedriver-win64.zip) dans le répertoire de travail de Python. Vous pouvez récupérer les versions stables sur [cette page](https://googlechromelabs.github.io/chrome-for-testing/#stable) (copier-coller le lien HTML dans la barre du navigateur pour télécharger le fichier).

`pip install selenium` in Terminal
### Configurer le navigateur Selenium (assurez-vous que ChromeDriver est installé)

In [4]:
driver = webdriver.Chrome()

## Création de l’URL à récupérer
On créé l’URL de la page qu’on souhaite récupérer. Pour faire une recherche dans Google Maps, il est préférable de mettre le nom de l’établissement, ainsi que les coordonnées géographiques de celui-ci.

In [5]:
base_url = "https://www.google.com/maps/search/"
place_info = "IUT+paris+rives+de+seine"
comp_url = "/@48.8489968,2.3125954,12z"

url = base_url + place_info + comp_url
url

'https://www.google.com/maps/search/IUT+paris+rives+de+seine/@48.8489968,2.3125954,12z'

## Premier lancement du navigateur
Pour rappel, **NE FERMEZ PAS LA FENETRE** mais cliquer sur Tout autoriser

In [6]:
driver.get(url)

## Fonction pour création de l'URL
On reprend la fonction vue dans la séance 1, avec une petite amélioration pour mettre le numéro de l'adresse en entier.

In [7]:
def f(v, fin = False):
    r = ""
    if not(pandas.isna(v)):
        if isinstance(v, float):
            r += str(int(v))
        else:
            r += str(v)
        if not(fin):
            r += "+"
    return r

### Code test de recherche & livraison

In [ ]:
# Liste pour stocker les résultats
site = []
livraison = []

# Limiter le DataFrame aux n premières lignes
df_subset = df.head(5)

# Boucle pour parcourir les données et rechercher les sites web
for i in range(df_subset.shape[0]):
    place_info = f(df["num"][i]) + f(df["let"][i]) + f(df["typ_voie"][i]) + f(df["lib_voie"][i], True)
    comp_url = "/@" + str(df["latitude"][i]) + "," + str(df["longitude"][i])
    url = base_url + place_info + comp_url
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    
    # Extraction du site web
    results = soup.select("a[aria-label*='Site Web']")
    
    # Si un résultat est trouvé, ajoutez le lien, sinon ajoutez None
    if results:
        href = results[0]["href"]
    else:
        href = None
    
    site.append(href)  # Ajouter le lien du site web ou None

    # Recherche de la mention "livraison" (à adapter selon la structure des sites)
    if "livraison" in html.lower() or "deliver" in html.lower() or "uber" in html.lower():
        livraison.append("Oui")
    else:
        livraison.append("Non")

    time.sleep(1)  # Respecter les délais pour éviter d'être bloqué

# Ajouter les résultats dans le sous-ensemble de DataFrame
df_subset["site"] = site
df_subset["livraison"] = livraison

# Sauvegarder le fichier avec les nouveaux résultats
df_subset.to_csv("data/BDCOM_2023_avec_site_et_livraison_test.csv", index=False)

print("Fichier sauvegardé pour les n premières adresses.")


C:\Users\leoun\AppData\Local\Temp\ipykernel_11896\3900287181.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset["site"] = site
C:\Users\leoun\AppData\Local\Temp\ipykernel_11896\3900287181.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset["livraison"] = livraison


Fichier sauvegardé pour les n premières adresses.


### Code de recherche web & livraison

In [ ]:
# Initialisation des listes pour stocker les résultats
site = []
livraison = []

for i in range(df.shape[0]):
    try:
        # Création de l'URL de recherche
        place_info = f(df["num"][i]) + f(df["let"][i]) + f(df["typ_voie"][i]) + f(df["lib_voie"][i], True)
        comp_url = "/@" + str(df["latitude"][i]) + "," + str(df["longitude"][i])
        url = base_url + place_info + comp_url
        
        # Ouvrir l'URL dans le navigateur Selenium
        driver.get(url)
        time.sleep(2)  # Pause pour charger la page

        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")

        # Extraction du site web
        results = soup.select("a[aria-label*='Site Web']")
        
        # Si un site web est trouvé, stocker le lien, sinon None
        if results:
            href = results[0]["href"]
        else:
            href = None
        
        site.append(href)

        # Recherche de la mention "livraison" dans la page
        if "livraison" in html.lower() or "deliver" in html.lower() or "uber" in html.lower():
            livraison.append("Oui")
        else:
            livraison.append("Non")

        # Affichage de la progression
        print(f"Traitement de l'adresse {i + 1}/{df.shape[0]} : {href}, Livraison: {livraison[-1]}")

    except Exception as e:
        print(f"Erreur à l'adresse {i + 1}/{df.shape[0]} : {str(e)}")
        site.append(None)
        livraison.append(None)

    time.sleep(1)  # Pause pour éviter le blocage de Google

# Ajout des résultats au DataFrame
df["site"] = site
df["livraison"] = livraison

# Sauvegarde du fichier enrichi
df.to_csv("data/BDCOM_2023_avec_site_et_livraison.csv", index=False)

print("Traitement terminé et fichier sauvegardé.")

Traitement de l'adresse 1/206723 : None, Livraison: Non
Traitement de l'adresse 2/206723 : None, Livraison: Non
Traitement de l'adresse 3/206723 : None, Livraison: Non
Traitement de l'adresse 4/206723 : None, Livraison: Non
Traitement de l'adresse 5/206723 : None, Livraison: Non
Traitement de l'adresse 6/206723 : None, Livraison: Non
Traitement de l'adresse 7/206723 : None, Livraison: Non
Traitement de l'adresse 8/206723 : None, Livraison: Non
Traitement de l'adresse 9/206723 : None, Livraison: Non
Traitement de l'adresse 10/206723 : None, Livraison: Non
Traitement de l'adresse 11/206723 : None, Livraison: Non
Traitement de l'adresse 12/206723 : None, Livraison: Non
Traitement de l'adresse 13/206723 : None, Livraison: Non
Traitement de l'adresse 14/206723 : None, Livraison: Non
Traitement de l'adresse 15/206723 : None, Livraison: Non
Traitement de l'adresse 16/206723 : None, Livraison: Non
Traitement de l'adresse 17/206723 : None, Livraison: Non
Traitement de l'adresse 18/206723 : None

KeyboardInterrupt: 